# 원-핫 인코딩 (One-Hot Encoding)
- One-Hot Encoding은 선택지의 개수만큼 Index를 가지며, 해당 되는 원소에는 1, 그 외에는 0으로 표시하는 Encoding 방식입니다.
  - EX) 수박,사과,딸기 일때, 딸기는 One-Hot Encoding으로 [0,0,1]로 표현합니다.
- One-Hot Encoding으로 표현한 벡터를 **One-Hot Vector**라고 합니다.

## One-Hot Encoding 사용하는 이유
- Class 별로 동일한 가중치를 갖기 때문에 One-Hot Vector가 적절합니다.
  - 만약, 1,2,3,4로 표현을 한다고 가정한다면 MSE를 구할 때, 4를 1로 잘못 예측한 것과 2를 1로 잘못 예측한 것이 오차 수준이 다릅니다.
  - 만약 이러한 방식으로 그냥 사용을 한다면, 4를 1로 예측한 것이 오차가 크기 때문에 정답(1)에 에러 작은 오답(2)가 더 가깝고, 에러가 큰 오답(4)는 관계가 더 적다고 모델이 인식합니다.
    - 하지만, 실제에서는 1이 아니라면 2나 4 모두 동일하게 잘못된 것이므로 이러한 인식은 옳지 않습니다.
  - 이러한 이유로 One-Hot Encoding이 적절합니다.
- 하지만, One-Hot Encoding은 이러한 장점 때문에 유사성을 알 수 없다는 단점도 존재합니다.
  - 클래스간의 관계를 알 수 없기 때문입니다.

## Class가 3개 이상인 상황의 분류 - SoftmaxRegression
1. Multi-class Classification
  - Softmax를 사용합니다.
    - Binary Classification에서도 두 선택지의 확률의 합이 1이었습니다. 이것을 확장하여 n개의 선택지가 된다면, n개 선택지의 확률의 합이 1인 함수를 사용하면 됩니다.
  - Softmax Function에서 $p_i$
    - $p_i = \frac{e^{z_i}}{Σ_{j=1}^{k}e^{z_j}}$ for i = 1,2,...,k
  - 실제 구현
    - Feature 4개로 3개의 클래스를 분류해야 하는 상황이라고 가정합니다.
      1. 3개의 클래스로 분류하므로 softmax는 3개의 확률값을 출력합니다.
      2. feature가 4개이고, class가 3개이므로 가중치는 12개가 필요합니다.
        - 따라서 W.shape == (3,4), x.shape == (4,1), b.shape == (3,1)입니다.

2. Softmax Regression에서 Loss Function
  - Cross Entropy Loss
    - 1개의 값에 대한 Loss Function
      - $Loss(W) = -Σ_{j=1}^{k}y_jlog(p_j)$
    - 전체에 대한 Loss Function
      - $Loss(W) = -\frac{1}{n}Σ_{i=1}^{n}Σ_{j=1}^{k}y_jlog(p_j)$
  - Softmax에서 사용한 CrossEntropyLoss도 BinaryClassification에서 이용한 CrossEntropyLoss와 본질적으로 동일합니다.

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

torch.manual_seed(1)

In [5]:
# soft max test
z = torch.FloatTensor([1,2,3])
H = F.softmax(z,dim = 0)
print(H) # softmax결과의 합이 1임을 알 수 있습니다.
print(H.sum())

tensor([0.0900, 0.2447, 0.6652])
tensor(1.)


In [12]:
# softmax test
z = torch.rand(3,5,requires_grad=True)

H = F.softmax(z,dim=1)
# 각 행별로 합이 1임을 알 수 있습니다. (5개의 클래스를 분류하는 것입니다.)
print(H)
print(H.sum(dim=1))

tensor([[0.2441, 0.1429, 0.2298, 0.2344, 0.1487],
        [0.1665, 0.2504, 0.2309, 0.1707, 0.1815],
        [0.2733, 0.1576, 0.2292, 0.2147, 0.1252]], grad_fn=<SoftmaxBackward0>)
tensor([1.0000, 1.0000, 1.0000], grad_fn=<SumBackward1>)


In [18]:
# 직접구현 시작

# y 는 정답 레이블을 나타냅니다.
y = torch.randint(5,(3,)).long()
print(y) # y 는 0~4 중의 한 개의 값을 갖는 원소들을 (3,)형태로 갖는 텐서입니다.

tensor([2, 1, 4])


In [21]:
# y(정답레이블)을 one-hot vector로 표현합니다.
#각 레이블에 대해 One-Hot Encoding을 진행합니다.
y_one_hot = torch.zeros_like(z) # z의 크기인 3 x 5 크기인 0 텐서를 생성합니다.
y_one_hot.scatter_(1,y.unsqueeze(1),1)
"""
scatter
first param - 값을 scatter할 차원을 표현 (각 열이 클래스를 의미하므로 1번째 차원(열)에 따라 뿌리게 설정)
second param - 값을 뿌릴 위치 표현 (y는 위에서처럼 정답 class를 의미하므로 해당 정답 클래스의 위치에 1을 뿌립니다.)
third param - 뿌릴 값을 표현 (one-hot vector를 만들어야 하므로 1을 뿌립니다.)
"""
y_one_hot

tensor([[0., 0., 1., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1.]])

In [23]:
# Loss Function을 구현합니다.
loss = (y_one_hot * -torch.log(H)).sum(dim=1).mean()
"""
y_one_hot * -torch.log(H).sum(dim=1) 으로 각 클래스의 오차를 합합니다.
.mean()으로 앞에서 구한 오차들의 평균을 구합니다.
"""
loss

tensor(1.6445, grad_fn=<MeanBackward0>)

## High-level Loss Implementation

### nll_loss
- Negative Log Likelihood를 계산하는 함수 (multi classification에서 주로 사용)
- nll_loss Input
  - first param : 모델의 예측값에 log를 취한 값, (F.log_softmax()는 softmax(예측)을 한 뒤, log를 취했으므로 적합)
  - second param : 실제 정답 클래스를 나타내는 텐서, (y 가 정답 레이블이었으므로 적합)
    - y_one_hot이 아닌, y를 사용하는 이유
      - nll_loss가 내부적으로 class index를 사용하여 loss를 계산하도록 설계되었기 때문입니다.
- nll_loss는 이 두가지 input바탕으로 negative log likelihood를 계산합니다.
  - 즉, model이 예측한 probability distribution과 실제 정답 label 사이의 차이를 측정합니다.
  

In [29]:
# Loss Function High-level implementation
## H = softmax이므로, -torch.log(softmax())가 됩니다. 이것은 -F.log_softmax()로 변환할 수 있습니다.
## 또한 nll_loss (negative log likelihood)를 이용하면, -F.log_softmax()이 외의 연산도 대체가 가능하므로 아래와 같이 표현할 수 있습니다.
loss = F.nll_loss(F.log_softmax(z,dim=1),y)
loss

tensor(1.6445, grad_fn=<NllLossBackward0>)

### cross_entropy
- Cross Entropy Loss를 계산하는 함수 (multi classification에서 주로 사용)
- cross_entropy input
  - first param : model's original output (original output before applying softmax like **z**)
  - second param : label ( it presents real class of each data like **y** )
- cross_entropy's detail
  - loss computation
    1. LogSoftmax : Compute log probability by applying log_softmax function on model's original output
    2. NLLLoss : Compute negative log likelihood by applying nll_loss function using log probability and label

In [30]:
loss = F.cross_entropy(z,y)
loss

tensor(1.6445, grad_fn=<NllLossBackward0>)

## Softmax Regression Implementation


In [31]:
# import

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(42)

In [32]:
#data
## feature : 4 , sample : 8, class : 3
x_train = [[1, 2, 1, 1],
            [2, 1, 3, 2],
            [3, 1, 3, 4],
            [4, 1, 5, 5],
            [1, 7, 5, 5],
            [1, 2, 5, 6],
            [1, 6, 6, 6],
            [1, 7, 7, 7]]
y_train = [2, 2, 2, 1, 1, 1, 0, 0]
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)

### low-level (directly implementation)

In [33]:
y_one_hot = torch.zeros((8,3))
y_one_hot.scatter_(1,y_train.unsqueeze(1),1)
y_one_hot

tensor([[0., 0., 1.],
        [0., 0., 1.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 1., 0.],
        [0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.]])

In [34]:
W = torch.zeros((4,1),requires_grad=True)
b = torch.zeros((1,3),requires_grad=True)
opt = optim.SGD([W,b],lr = 0.1)

In [40]:
epochs = 1000
for epoch in range(epochs):
  output = F.softmax(x_train.matmul(W)+b,dim=1)
  loss = (y_one_hot * -torch.log(output)).sum(dim=1).mean()

  opt.zero_grad()
  loss.backward()
  opt.step()

  if epoch % 100 == 0 :
    print('Epoch {:4d}/{} loss: {:.6f}'.format(epoch, epochs, loss.item()))

Epoch    0/1000 loss: 1.098612
Epoch  100/1000 loss: 1.082238
Epoch  200/1000 loss: 1.082196
Epoch  300/1000 loss: 1.082196
Epoch  400/1000 loss: 1.082196
Epoch  500/1000 loss: 1.082196
Epoch  600/1000 loss: 1.082196
Epoch  700/1000 loss: 1.082196
Epoch  800/1000 loss: 1.082195
Epoch  900/1000 loss: 1.082195


## High-levle